In [ ]:
!pip install pymysql > /dev/null

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [ ]:
import json 
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql 
conn = pymysql.connect(
     host = config['host'],
     user = config['user'],
     password = config['password'],
     database = config['database'],
     port = config['port'])

In [ ]:
#위에 귀찮으면

import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)    # dictionary unpacking

In [ ]:
import random
random.seed(2021)

# 1.테이블 생성

- product 테이블 생성

In [ ]:
sql_products = '''
CREATE  TABLE  if not exists products(
 pid INT  PRIMARY KEY AUTO_INCREMENT,
 pname varchar(40) not null,
 pprice INT UNSIGNED not null,
 pcategory VARCHAR(40) not null,
 pcost INT UNSIGNED  not null
) AUTO_INCREMENT=11;
'''

In [ ]:
cur = conn.cursor()
cur.execute(sql_products)

0

- product 상품정보 입력

In [ ]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품', '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV', '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리', '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [ ]:
for item in camping_goods:
    price = random.randint(100, 999) * 100
    rate = random.randint(85, 95)
    cost = int(round(price * rate) / 1000) * 10
    print(item, price, cost, '캠핑용품')

텐트 76300 71720 캠핑용품
타프 42100 36620 캠핑용품
천막 34600 32520 캠핑용품
침낭 38800 33750 캠핑용품
매트 14300 13150 캠핑용품
랜턴 85600 73610 캠핑용품
취사용품 37100 33760 캠핑용품
아이스박스 55200 47470 캠핑용품
보조배터리 70100 60280 캠핑용품
워터저그 61900 52610 캠핑용품


In [ ]:
sql_products = "insert into products(pname, pprice, pcategory, pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100, 999) * 100
        rate = random.randint(85, 95)
        cost = int(round(price * rate) / 1000) * 10
        cur.execute(sql_products, (item, price, category_names[i], cost))
conn.commit()

- sales 테이블생성

In [ ]:
cur = conn.cursor()
sales = '''
create table if not exists sales (
 sid INT primary key AUTO_INCREMENT ,
 sdate date not null,
 scompany varchar(40) not null,
 spid INT not null,
 sunit INT not null,
 FOREIGN KEY(spid) references products(pid)
  ) AUTO_INCREMENT=101;
'''

In [ ]:
cur = conn.cursor()
cur.execute(sales)

0

In [ ]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
for i in range(10):
    date = f'2020-{i}-{random.randint(1,31)}'
date

'2020-1-21'

In [ ]:
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1, day)}'
    print(date)

2020-1-13
2020-2-20
2020-3-15
2020-4-17
2020-5-28
2020-6-2
2020-7-28
2020-8-14
2020-9-6
2020-10-2
2020-11-4
2020-12-31


In [ ]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']
for i in range(10):
    date = f'2020-{month}-{random.randint(1,day)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-12-4 표준협회 24 10
2020-12-24 생산성본부 13 7
2020-12-12 표준협회 32 16
2020-12-8 표준협회 37 12
2020-12-7 생산성본부 29 3
2020-12-13 한경아카데미 18 14
2020-12-21 표준협회 28 18
2020-12-21 멀티캠퍼스 25 11
2020-12-3 한경아카데미 12 7
2020-12-17 테크브루 20 11


In [ ]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

#두 테이블을 Join 한 View 생성

In [ ]:

sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [ ]:
cur = conn.cursor()
cur.execute(sql)

In [ ]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [ ]:
results

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,104,2020-01-02,멀티캠퍼스,29,방진매트,자동차용품,80800,20,74330,1616000,129400
1,107,2020-01-08,생산성본부,13,천막,캠핑용품,78800,20,67760,1576000,220800
2,110,2020-01-12,표준협회,38,스텝퍼,헬스용품,18700,18,17010,336600,30420
3,109,2020-01-16,한경아카데미,25,자동차TV,자동차용품,81200,9,74700,730800,58500
4,106,2020-01-17,표준협회,37,트램펄린,헬스용품,36600,3,32940,109800,10980


In [ ]:
del df[3]
del df[0]

In [ ]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-02,멀티캠퍼스,방진매트,자동차용품,80800,20,74330,1616000,129400
1,2020-01-08,생산성본부,천막,캠핑용품,78800,20,67760,1576000,220800
2,2020-01-12,표준협회,스텝퍼,헬스용품,18700,18,17010,336600,30420
3,2020-01-16,한경아카데미,자동차TV,자동차용품,81200,9,74700,730800,58500
4,2020-01-17,표준협회,트램펄린,헬스용품,36600,3,32940,109800,10980


In [ ]:
df.to_csv('sales_book.csv', index=False)

마무리

In [ ]:
cur.close()
conn.close()